# Стохастический градиентный и координатный спуски (практика)

In [119]:
# Загрузим требуемые библиотеки
import numpy as np
import pandas as pd
import sklearn
import sympy
from sympy import *
from sklearn.linear_model import *
from sklearn.model_selection import *
from sklearn.metrics import *


In [120]:
# Загрузим исходный датасет
df = pd.read_csv('data\Advertising.csv',index_col=0)
df.head()

,TV,radio,newspaper,sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9


In [121]:
# проверим наличие пропусков
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 1 to 200
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         200 non-null    float64
 1   radio      200 non-null    float64
 2   newspaper  200 non-null    float64
 3   sales      200 non-null    float64
dtypes: float64(4)
memory usage: 7.8 KB


*Пропуски в данных отсутствуют*

In [122]:
# выделим предикторы и целевую переменную
X = df[['TV','radio','newspaper']]
y = df['sales']

_______________________

### Координатный спуск

In [123]:
# Добавим единичный столбец для того, чтобы у нас был свободный коэффициент в уравнении регрессии:
X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])
y = y.values.reshape(-1, 1)
print(X.shape, y.shape)

(200, 4) (200, 1)


In [124]:
# Нормализуем данные
X = X / np.sqrt(np.sum(np.square(X), axis=0))

In [125]:
# Реализуем алгоритм координатного спуска:
# num_iters = количество итераций
# m = количество строк в матрице X
# n = количество столбцов в матрице X
# w = вектор размера nx1, состояющий из нулей
num_iters = 1000
n =X.shape[1]
m = X.shape[0]
w = np.zeros((n,1))

for i in range(num_iters):
    for k in range(n):
        # Вычисляем прогноз без k-ого фактора
        h = (X[:,0:k] @ w[0:k]) + (X[:,k+1:] @ w[k+1:])
        # Обновляем новое значение k-ого коэффициента
        w[k] =  (X[:,k].T @ (y - h))
        # Вычисляем функцию потерь
        cost = sum(((X @ w) - y) ** 2)/(len(y))
w

array([[ 41.56217205],
       [110.13144155],
       [ 73.52860638],
       [ -0.55006384]])

In [126]:
# Сравним результаты с реализацией линейной регрессии из библиотеки sklearn:
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=False)
model.fit(X, y)
print(model.coef_)

[[ 41.56217205 110.13144155  73.52860638  -0.55006384]]


In [127]:
# получим вектор прогнозов
y_pred = X @ w
# рассчитаем среднеквадратичную ошибку для координатного спуска:
print ('mse= ', mean_squared_error(y, y_pred))
print ('mae= ', mean_absolute_error(y, y_pred))

mse=  2.784126314510936
mae=  1.252011229687068


_________________________________

### Стохастический градиентный спуск

In [128]:
import numpy as np
from sklearn import preprocessing
# minmax_scaler = preprocessing.MinMaxScaler(feature_range=(0,1))

# Отмасштабируем  столбцы исходной матрицы X:
df = pd.read_csv('data\Advertising.csv',index_col=0)
X = df[['TV','radio','newspaper']]
y = np.array(df['sales'])
X=(X-np.mean(X, axis=0))/np.std(X, axis=0)


In [129]:
y

array([22.1, 10.4,  9.3, 18.5, 12.9,  7.2, 11.8, 13.2,  4.8, 10.6,  8.6,
       17.4,  9.2,  9.7, 19. , 22.4, 12.5, 24.4, 11.3, 14.6, 18. , 12.5,
        5.6, 15.5,  9.7, 12. , 15. , 15.9, 18.9, 10.5, 21.4, 11.9,  9.6,
       17.4,  9.5, 12.8, 25.4, 14.7, 10.1, 21.5, 16.6, 17.1, 20.7, 12.9,
        8.5, 14.9, 10.6, 23.2, 14.8,  9.7, 11.4, 10.7, 22.6, 21.2, 20.2,
       23.7,  5.5, 13.2, 23.8, 18.4,  8.1, 24.2, 15.7, 14. , 18. ,  9.3,
        9.5, 13.4, 18.9, 22.3, 18.3, 12.4,  8.8, 11. , 17. ,  8.7,  6.9,
       14.2,  5.3, 11. , 11.8, 12.3, 11.3, 13.6, 21.7, 15.2, 12. , 16. ,
       12.9, 16.7, 11.2,  7.3, 19.4, 22.2, 11.5, 16.9, 11.7, 15.5, 25.4,
       17.2, 11.7, 23.8, 14.8, 14.7, 20.7, 19.2,  7.2,  8.7,  5.3, 19.8,
       13.4, 21.8, 14.1, 15.9, 14.6, 12.6, 12.2,  9.4, 15.9,  6.6, 15.5,
        7. , 11.6, 15.2, 19.7, 10.6,  6.6,  8.8, 24.7,  9.7,  1.6, 12.7,
        5.7, 19.6, 10.8, 11.6,  9.5, 20.8,  9.6, 20.7, 10.9, 19.2, 20.1,
       10.4, 11.4, 10.3, 13.2, 25.4, 10.9, 10.1, 16

In [130]:
X

,TV,radio,newspaper
1,0.969852,0.981522,1.778945
2,-1.197376,1.082808,0.669579
3,-1.516155,1.528463,1.783549
4,0.052050,1.217855,1.286405
5,0.394182,-0.841614,1.281802
...,...,...,...
196,-1.270941,-1.321031,-0.771217
197,-0.617035,-1.240003,-1.033598
198,0.349810,-0.942899,-1.111852
199,1.594565,1.265121,1.640850


In [131]:
# Добавим единичный столбец для того, чтобы у нас был свободный коэффициент в уравнении регрессии:
X = np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])
X

array([[ 1.00000000e+00,  9.69852266e-01,  9.81522472e-01,
         1.77894547e+00],
       [ 1.00000000e+00, -1.19737623e+00,  1.08280781e+00,
         6.69578760e-01],
       [ 1.00000000e+00, -1.51615499e+00,  1.52846331e+00,
         1.78354865e+00],
       [ 1.00000000e+00,  5.20496822e-02,  1.21785493e+00,
         1.28640506e+00],
       [ 1.00000000e+00,  3.94182198e-01, -8.41613655e-01,
         1.28180188e+00],
       [ 1.00000000e+00, -1.61540845e+00,  1.73103399e+00,
         2.04592999e+00],
       [ 1.00000000e+00, -1.04557682e+00,  6.43904671e-01,
        -3.24708413e-01],
       [ 1.00000000e+00, -3.13436589e-01, -2.47406325e-01,
        -8.72486994e-01],
       [ 1.00000000e+00, -1.61657614e+00, -1.42906863e+00,
        -1.36042422e+00],
       [ 1.00000000e+00,  6.16042873e-01, -1.39530685e+00,
        -4.30581584e-01],
       [ 1.00000000e+00, -9.45155670e-01, -1.17923146e+00,
        -2.92486143e-01],
       [ 1.00000000e+00,  7.90028350e-01,  4.96973404e-02,
      

In [132]:
# Создадим функцию mse_error для вычисления среднеквадратичной ошибки
def mse_error (A, B):
    mse = ((A - B)**2).mean(axis=0)
    return mse

In [133]:
# Сделаем наивный прогноз: предскажем продажи средним значением
x_m = df['sales'].mean()
y_pred = df['sales'] = df['sales'].apply(lambda x: x_m)
y_pred

1      14.0225
2      14.0225
3      14.0225
4      14.0225
5      14.0225
        ...   
196    14.0225
197    14.0225
198    14.0225
199    14.0225
200    14.0225
Name: sales, Length: 200, dtype: float64

In [134]:
# рассчитаем среднеквадратичную ошибку для этого прогноза:
mse_error (y, y_pred )

27.085743750000002

In [135]:
# проверим полученный трезультат для среднеквадратичной ошибки с помощью библиотеки sklearn:
from sklearn.metrics import mean_squared_error
mean_squared_error(y, y_pred)

27.085743750000002

In [136]:
# Создадим функцию, которая может по матрице предикторов *X* и вектору весов линейной модели *w* получить вектор прогнозов
def lin_pred (X, w):
    y_pred = np.dot(X, w)
    return y_pred

In [137]:
# получим вектор прогнозов
lin_pred (X, w)
#X @ w

array([[ 2.19564846e+02],
       [-1.10575596e+01],
       [-1.40094509e+01],
       [ 1.36134050e+02],
       [ 2.23862737e+01],
       [-1.01899642e+01],
       [-2.60646867e+01],
       [-1.06685700e+01],
       [-2.40802793e+02],
       [ 7.04974099e+00],
       [-1.49075544e+02],
       [ 1.32895668e+02],
       [-5.90568819e+01],
       [-9.93281236e+01],
       [ 1.62388430e+02],
       [ 2.24506310e+02],
       [ 3.75619537e+00],
       [ 2.94812341e+02],
       [-7.19553548e+01],
       [ 4.53410199e+01],
       [ 1.54773186e+02],
       [ 6.77570627e+01],
       [-1.67602002e+02],
       [ 1.14572153e+02],
       [-1.20051510e+02],
       [ 9.27072631e+01],
       [ 6.66577725e+01],
       [ 1.28837426e+02],
       [ 1.91660649e+02],
       [-9.30667865e+01],
       [ 2.53816759e+02],
       [-3.16627680e+01],
       [-1.30577198e+02],
       [ 1.78586661e+02],
       [-1.32430435e+02],
       [ 1.31715296e+02],
       [ 2.98304380e+02],
       [ 7.79098116e+01],
       [-7.5

In [138]:
# создадим функцию для реализации шага стохастического градиентного спуска
def stoch_grad_step (X, y, w, train_ind, eta=0.01):
    grad_array = np.zeros(np.shape(X)[1])
    for i in range(0, np.shape(X)[1]):
        grad_array[i] = 2 * X[train_ind][i] *(sum(w * X[train_ind]) - y[train_ind]) / np.shape(X)[0]
    return w - eta * grad_array

In [139]:
# создадим функцию для реализации стохастического градиентного спуска
def stochastic_gradient_descent(X, y, w_init, max_iter=10000, min_weight_dist=1e-8, seed=42):
    # расстояние между векторами весов на соседних итерациях большим числом.
    weight_dist = np.inf
    # Инициализируем вектор весов
    w = w_init
    # ошибки на каждой итерации
    errors = []
    # Счетчик итераций
    iter_num = 0
    # для воспроизводимости последовательности случайных чисел используем seed.
    np.random.seed(seed)
    while weight_dist > min_weight_dist and iter_num < max_iter:
        # случайный индекс объекта обучающей выборки
        random_ind = np.random.randint(X.shape[0])
        # шаг стохастического градиентного спуска
        w_new = stoch_grad_step (X, y, w, random_ind)
        # дистанция между старым и новым значениями вектора весов
        weight_dist = np.linalg.norm(w_new - w)
        w = w_new
        # ошибка предсказания
        errors.append(mse_error(y, lin_pred(X, w)))
        iter_num +=1
    return w, errors

In [140]:
# Запустим 100000 итераций стохастического градиентного спуска, укажем вектор начальных весов, состоящий из нулей.
stoch_grad_desc_weights, stoch_errors_by_iter = stochastic_gradient_descent(X, y, np.zeros(np.shape(X)[1]), max_iter=100000)
stoch_errors_by_iter

[223.66538899863156,
 223.63640485031922,
 223.55491009541456,
 223.4807624515579,
 223.47256426508545,
 223.44655231890474,
 223.39175144920213,
 223.3214521757322,
 223.2707462145908,
 223.2585746385283,
 223.20263393030626,
 223.1464246501442,
 223.1182865596016,
 223.05639626029313,
 223.01523254529158,
 222.99035057946475,
 222.98741420324143,
 222.96901770985096,
 222.87192976119704,
 222.84459487822633,
 222.78857273967887,
 222.76972781414096,
 222.72086214383265,
 222.70490416801397,
 222.69156394466313,
 222.63455887297178,
 222.56452772523568,
 222.52385945121242,
 222.49253589220314,
 222.45710958268128,
 222.42121099760504,
 222.37014375278872,
 222.26235109722083,
 222.21264479327613,
 222.1557583870522,
 222.08211118403963,
 222.07508646895414,
 222.06806832747162,
 222.0392433526464,
 222.03223515214347,
 222.00362710089578,
 221.99280835153607,
 221.91322993552527,
 221.87989171544683,
 221.8770161171395,
 221.84844887405282,
 221.81889665012451,
 221.74912825062972,
 

In [141]:
# Построим график зависимости ошибки от номера итерации
import plotly.graph_objects as go
x = np.arange(len(stoch_errors_by_iter))
fig = go.Figure(data=go.Scatter(x=x, y=stoch_errors_by_iter, ))

fig.update_layout(title='Зависимость ошибки mse от номера итерации',
                   xaxis_title='номер итерации',
                   yaxis_title='mse')
fig.show()

In [142]:
# Выведем вектор весов, к которому сошелся метод
stoch_grad_desc_weights

array([ 1.40190566e+01,  3.91069256e+00,  2.78209808e+00, -8.10462217e-03])

In [143]:
# Выведем среднеквадратичную ошибку на последней итерации
stoch_errors_by_iter[-1]

2.7844125884067044

In [144]:
# получим вектор прогнозов
y_pred = X @ w
# рассчитаем среднеквадратичную ошибку для стохастического градиентного спуска:
print ('mse= ', mean_squared_error(y, lin_pred (X, stoch_grad_desc_weights)))
print ('mae= ', mean_absolute_error(y, lin_pred (X, stoch_grad_desc_weights)))

mse=  2.7844125884067044
mae=  1.2536250176349868


***Вывод:*** *значения метрик  MSE и MAE для обоих методов практически совпадают (до 2-го знака после запятой).*

_________________________